#  TOKEN is a MASK (Few-shot example) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uds-lsv/TOKEN-is-a-MASK/blob/main/example_few-shot.ipynb) 

In [26]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

name, driver_version, memory.total [MiB]
Tesla T4, 460.32.03, 15109 MiB


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13202587730578660700
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14444920832
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1209427256583303605
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
!pip install -q datasets
!pip install -q  transformers
!pip install -q seqeval[cpu]
!pip install -q -U spacy
!python -q -m spacy download en_core_web_sm
#!pip install -q allennlp==1.0.0 allennlp-models==1.0.0
#!pip install -q gensim

     |████████████████████████████████| 365 kB 5.2 MB/s 
     |████████████████████████████████| 115 kB 64.3 MB/s 
     |████████████████████████████████| 212 kB 59.9 MB/s 
     |████████████████████████████████| 141 kB 60.3 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
     |████████████████████████████████| 596 kB 56.6 MB/s 
     |████████████████████████████████| 127 kB 68.4 MB/s 
     |████████████████████████████████| 4.7 MB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 49.0 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 1.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [27]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import logging
import pandas as pd
from datasets import Dataset
from collections import Counter, defaultdict
from time import time
import string
from tqdm import tqdm
import spacy
import gensim.downloader as api
from scipy.special import softmax
import pickle

from seqeval.metrics import f1_score, classification_report
from sklearn.metrics import classification_report as sk_classification_report
from sklearn.metrics import f1_score as sk_f1_score

# from allennlp.predictors.predictor import Predictor
# import allennlp_models.structured_prediction

In [28]:
nlp = spacy.load("en_core_web_sm")
#word_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
device = 'cuda'
model_name = 'bert-large-cased'
model = BertForMaskedLM.from_pretrained(model_name).to(device)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Dataset to Evaluate on (in CoNLL Format)

In [29]:
!git clone https://github.com/uds-lsv/TOKEN-is-a-MASK


Cloning into 'TOKEN-is-a-MASK'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 94 (delta 36), reused 32 (delta 6), pack-reused 0
Unpacking objects: 100% (94/94), done.


In [30]:
data_path = 'TOKEN-is-a-MASK/data/conll03_100/'
output_path = 'TOKEN-is-a-MASK/output/'

In [31]:
def get_dataset(file):
  df = pd.read_csv(file, sep=" ", header=None, usecols=[0, 3], 
                   skip_blank_lines=False)
  df.columns = ['token', 'tag']
  df['tag'] = df['tag'].fillna('O')
  df.token[df.token == ' '] = '"'
  dataset = Dataset.from_pandas(df)

  return dataset

def get_pred_dataset(file):
  df = pd.read_csv(file, sep=" ", header=None, skip_blank_lines=False,
                   quoting=3)
  df.rename(columns={0: 'token', 1: 'tag'}, inplace=True)
  df['tag'] = df['tag'].fillna('O')
  dataset = Dataset.from_pandas(df)

  return dataset

def get_dataset_native_py(file):
  tokens = []
  tags = []
  with open(file) as f:
      for line in f:
        _list = line.split()
        if len(_list) == 4:
          tokens.append(_list[0])
          tags.append(_list[-1])
        elif len(_list) == 0:
          tokens.append(None)
          tags.append('O')
        else:
          raise print(_list)

  df = pd.DataFrame({'token': tokens, 'tag': tags })
  df['tag'] = df['tag'].fillna('O')
  df.token[df.token == ' '] = '"'
  dataset = Dataset.from_pandas(df)

  return dataset

#pred_test_dataset =  get_pred_dataset_native_py(output_path+'test_predictions.txt')
pred_test_dataset = get_pred_dataset(output_path+'test_predictions.txt')

test_dataset = get_dataset_native_py(data_path+'test.txt')

In [32]:
test_dataset.to_pandas().columns

Index(['token', 'tag'], dtype='object')

# Using POS 

In [33]:
ent_pos = 'PROPN'
def get_pos(seq):
  #print(seq)
  if None in seq:
    seq.remove(None)

  text = " ".join(seq)  
  doc = nlp(text)
  token_tags = []
  for token in doc:
    if token.pos_ == 'SPACE':
      token_tags.append([' ', token.pos_])
    elif len(token) == 1:
      token_tags.append([token.text, 'O'])
    else:
      token_tags.append([token.text, token.pos_])
  
  seq_tags = []
  counter = 0
  running_token = ''
  running_tags = []
  for t, pos in token_tags:
    if t == seq[counter]:
      seq_tags.append([t, pos])
      counter += 1
    else:
      running_token += t
      running_tags += [pos]
      if running_token == seq[counter]:
        if ent_pos in running_tags:
          _pos = ent_pos
        else:
          _pos = running_tags[0]
        seq_tags.append([running_token, _pos])
        counter += 1
        running_token = ''
        running_tags = []

  if len(seq) > len(seq_tags):
    seq = seq[:len(seq_tags)]
  assert seq == [t for t, _ in seq_tags]

  for i in range(len(seq)):
    if seq_tags[i][1] == ent_pos:
      seq_tags[i][0] = seq_tags[i][0].capitalize()
      seq[i] = seq[i].capitalize()
  seq[0] = seq[0].capitalize()
  return seq, seq_tags

In [34]:
i = 16
j = 22
seq = test_dataset[i: j]['token']
get_pos(seq)

(['Al-ain', ',', 'United', 'Arab', 'Emirates', '1996-12-06'],
 [['Al-ain', 'PROPN'],
  [',', 'O'],
  ['United', 'PROPN'],
  ['Arab', 'PROPN'],
  ['Emirates', 'PROPN'],
  ['1996-12-06', 'NUM']])

In [35]:
i = 33697 #33718
j = 33722
seq = test_dataset[i: j]['token']
get_pos(seq)

(['There',
  'are',
  '500',
  'million',
  'messages',
  'transmitted',
  'through',
  'the',
  'Internet',
  'everyday',
  ',',
  '"',
  'said',
  'Tim',
  'Casey',
  'of',
  'the'],
 [['There', 'PRON'],
  ['are', 'VERB'],
  ['500', 'NUM'],
  ['million', 'NUM'],
  ['messages', 'NOUN'],
  ['transmitted', 'VERB'],
  ['through', 'ADP'],
  ['the', 'DET'],
  ['Internet', 'NOUN'],
  ['everyday', 'ADV'],
  [',', 'O'],
  ['"', 'O'],
  ['said', 'VERB'],
  ['Tim', 'PROPN'],
  ['Casey', 'PROPN'],
  ['of', 'ADP'],
  ['the', 'DET']])

In [36]:
def get_labels(text, template_text, ner_words_):
  #print(text)
  #print(template_text)
  tokenized_text = tokenizer.tokenize(text)
  tokenized_template = tokenizer.tokenize(template_text)
  _tokenized_text = tokenized_text + tokenized_template
  masked_ids = [i for i, x in enumerate(_tokenized_text) if x == "[MASK]"]
  indexed_tokens = tokenizer.convert_tokens_to_ids(_tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens]).to(device)

  segments_ids_text = [0] * len(tokenized_text)
  segments_ids_template = [1] * len(tokenized_template)
  segments_ids = segments_ids_text + segments_ids_template
  segments_tensors = torch.tensor([segments_ids]).to(device)

  outputs = model(tokens_tensor, token_type_ids=segments_tensors)
  predictions = outputs[0]
  prob = torch.softmax(predictions[0, masked_ids], dim=-1)

  labels_dic = {}
  for tag, ids in ner_words_.items():
      labels_dic[tag] = prob[0, ids].max().item()
  #print(labels_dic)
  pred_label = max(labels_dic, key=labels_dic.get)
  pred_prob = labels_dic[pred_label]
  return pred_label, pred_prob

In [37]:
def classifier(seq, pos_tags):
  text_seq = ["[CLS]"] + seq + ["[SEP]"]
  text =  ' '.join(text_seq)

  len_pos = len(pos_tags)
  pred_tags = []
  _entity = '' 
  counter = 0
  for i, (t, pos) in enumerate(pos_tags):
    if pos not in ent_pos:
      pred_tags.append('O')
      _entity = ''
      counter = 0
      continue

    _entity += ' ' + t
    counter += 1
    if i+1 < len_pos and pos_tags[i+1][1] in ent_pos:
      continue
    else:
      pred_tag = 'O'
      pred_prob = 0.2
      _ner_words = ner_words
      _templates = templates[:2]
      if pos in ent_pos[1:]:
        pred_prob = 0.3
        #_ner_words = adv_adj_words
        _templates = templates[2:]
      for template in _templates:
        template_text = _entity +  template
        _pred_label, _pred_prob = get_labels(text, template_text, _ner_words)
        if _pred_prob > pred_prob:
          pred_prob = _pred_prob
          pred_tag = _pred_label
      pred_tags += [pred_tag] * counter

  assert len(pred_tags) == len(seq), print(seq, len(pred_tags), len(seq))

  bio_pred_tags = []
  for i, tag in enumerate(pred_tags):
    if tag == 'O':
      bio_pred_tags += [tag]
      continue
    if len(bio_pred_tags) == 0:
      bio_pred_tags += ['B-'+tag]

    elif tag in bio_pred_tags[i-1]:
      bio_pred_tags += ['I-'+tag]
    else:
      bio_pred_tags += ['B-'+tag]

  return bio_pred_tags

In [38]:
def flatten(_list):
  return [item for sublist in _list for item in sublist]


def analyze_predictions(gt_dataset, pred_dataset):
  gt_dataset_tokens = gt_dataset['token']
  gt_dataset_tags = gt_dataset['tag']

  pred_dataset_tokens = pred_dataset['token']
  pred_dataset_pred_tags = pred_dataset['tag']
  #pred_dataset_probs = pred_dataset['pred_probs']

  seqs = []
  gt_tags = []
  pred_tags = []

  seq = []
  gt_tag = []
  pred_tag = []

  
  num_example = 0
  for i, token in enumerate(gt_dataset_tokens):
    assert token == pred_dataset_tokens[i], print(i, token, pred_dataset_tokens[i])
    _gt_tag = gt_dataset_tags[i]
    _pred_tag = pred_dataset_pred_tags[i]
    #_prob = pred_dataset_probs[i]
    if token == None:
      if len(seq) > 0:
        seq, pos_tags = get_pos(seq)
        #print(pos_tags)
        pred_tag_template = classifier(seq, pos_tags) 
        #print(pred_tag_template)
        #print(pred_tag)
        #print(gt_tag)
        final_seq_tag = []
        for k, label in enumerate(pred_tag_template):
          #if pred_tag[k][:2] in ['PERSON', 'DATE']:
           # final_seq_tag.append(pred_tag[k])
          #elif  label[2:] in ['PERSON', 'PROFESSION']: #'LOC' 'PHONE'
          #  final_seq_tag.append(label[:2]+'PROFESSION')
          if  label[2:] in ['LOC','ORG', 'PER']:
            final_seq_tag.append(label)
          else:
            final_seq_tag.append(pred_tag[k])
        #print(final_seq_tag)

        if len(gt_tag) == len(final_seq_tag):
          seqs.append(seq)
          gt_tags.append(gt_tag)
          pred_tags.append(final_seq_tag)

        seq = []
        gt_tag = []
        pred_tag = []
        wrong_pred = []
        num_example += 1

        
        if num_example % 200 == 0:
            print(classification_report(gt_tags, pred_tags))
      else:
        continue
    else:
      seq += [token]
      gt_tag += [_gt_tag]
      pred_tag += [_pred_tag]


  print(classification_report(gt_tags, pred_tags))
  print(f1_score(gt_tags, pred_tags))
#  return confusion_matrix(flatten(gt_tags), flatten(pred_tags), labels=labels),\
#   wrong_predictions, wrong_probs, correct_pobs

In [39]:
def get_word_ids(words):
  ids = []
  for w in words:
    _id = tokenizer.convert_tokens_to_ids(w)
    ids.append(_id)
  
  assert len(ids) == len(words)
  return ids

init_loc_words = ['location', 'city', 'country', 'region', 'area',
             'province', 'state', 'town', 'downtown']
init_per_words = ['person', 'man', 'woman', 'boy', 'girl', 'human', 'someone', 'kid']
init_org_words = ['organization', 'community', 'commision', 'department',
             'association', 'council', 'company', 'union', 'team', 'group', 'firm']
init_misc_words = ['citizenship', 'nationality', 'ethnicity', 'cup' , 'game']

ner_words_text = {'ORG': init_org_words, #'MISC': init_misc_words,
            'PER': init_per_words, 'LOC': init_loc_words}

ner_words = {k: get_word_ids(v) for k, v in ner_words_text.items()}   

templates = [' is a [MASK] . [SEP]', ' is an [MASK] . [SEP]']
#templates = [' is an instance of a [MASK] . [SEP]', ' is an instance of an [MASK] . [SEP]']

#templates = [' denotes a [MASK] . [SEP]', ' denotes an [MASK] . [SEP]']

#templates = ['  would be a [MASK] . [SEP]', '  would be an [MASK] . [SEP]']

#templates = ['  is well known to be  a [MASK] . [SEP]', '  is well known to be an  [MASK] . [SEP]']

#templates = ['   is a common [MASK] known to many people. [SEP]']


#templates = [' [MASK] . [SEP]', ' [MASK] . [SEP]']

ner_words

{'ORG': [2369, 1661, 100, 2853, 3852, 3193, 1419, 3779, 1264, 1372, 3016],
 'PER': [1825, 1299, 1590, 2298, 1873, 1769, 1800, 5102],
 'LOC': [2450, 1331, 1583, 1805, 1298, 3199, 1352, 1411, 5215]}

In [40]:
analyze_predictions(test_dataset, pred_test_dataset)

              precision    recall  f1-score   support

         LOC       0.68      0.82      0.74       130
         ORG       0.34      0.41      0.37        49
         PER       0.87      0.80      0.83       193

   micro avg       0.71      0.75      0.73       372
   macro avg       0.63      0.67      0.65       372
weighted avg       0.73      0.75      0.74       372

              precision    recall  f1-score   support

         LOC       0.67      0.74      0.70       266
         ORG       0.28      0.28      0.28       102
         PER       0.90      0.85      0.87       425

   micro avg       0.73      0.74      0.74       793
   macro avg       0.61      0.63      0.62       793
weighted avg       0.74      0.74      0.74       793

              precision    recall  f1-score   support

         LOC       0.60      0.71      0.65       309
         ORG       0.33      0.20      0.25       266
         PER       0.86      0.85      0.85       451

   micro avg       0